In [22]:
import numpy as np
import pandas as pd
import random

 # self-created helping module:
from RL_Helper import helpers as hlp


# RL-based Recsys and Evaluation

In [23]:
# load data
corrected_policy = pd.read_csv('C:/~your_path~/RecSys/obtained_data/corrected_policy_maj_vote_pol_eps05.csv')

In [24]:
corrected_policy_grades = pd.read_csv('C:/~your_path~/RecSys/obtained_data/corrected_policy_mean_grades_pol_eps05.csv')

In [25]:
corrected_policy_complete = pd.read_csv('C:~your_path~/RecSys/obtained_data/corrected_policy_complete_eps05.csv')

In [26]:
# inspect data
corrected_policy.tail()

,CourseID,Rank
63,T208015,0.0
64,T205000,0.0
65,T206059,0.0
66,T214006,0.0
67,T208033,0.0


In [27]:
corrected_policy_grades.tail()

,CourseID,Rank
63,T205023,0.0
64,T206059,0.0
65,T208015,0.0
66,T214006,0.0
67,T208033,0.0


In [28]:
test_df_NaN = pd.read_csv('C:~your_path~/RecSys/obtained_data/test_df_NaN.csv') # both programs
test_df_NaN_pol = pd.read_csv('C:~your_path~/RecSys/obtained_data/test_df_NaN_pol.csv')
#courses = pd.read_csv('obtained_data/all_courses.csv')

In [71]:
## initial data set that is needed for validation
test_df = pd.read_csv('C:/Users/Christina Rollheiser/Desktop/RecSys/obtained_data/test_df.csv')
test_df_pol = pd.read_csv('C:/Users/Christina Rollheiser/Desktop/RecSys/obtained_data/test_df_pol.csv')

In [30]:
test_df_NaN.tail(11)

,CourseID,StudentID,Grades
4649,T207002,172,4.25
4650,T206060,172,NaN
4651,T207004,172,NaN
4652,T207036,172,NaN
4653,J2D035,172,NaN
4654,T207002,172,NaN
4655,5869,172,NaN
4656,5870,172,NaN
4657,T206057,172,NaN
4658,T206024,172,NaN


## Core Functions

In [32]:
def bandit(candidates, x):# x is the number of recommendations to be obtained 
    random.seed(32)
    chosen_arms = pd.DataFrame()
    candidates = candidates.sort_values(by = 'Rank', ascending = False)
    df_temp = pd.DataFrame({ # adding the first 50 % of x recommendations needed to the list, as candidates are sorted in descending order
        'CourseID':candidates.CourseID[: int(x / 2)],
        'Rank':candidates.Rank[: int(x / 2)]
    })
    chosen_arms = pd.concat([chosen_arms, df_temp])
    count = int(x / 2)
    candidates = candidates[~candidates.CourseID.isin(chosen_arms.CourseID)] # filter candidates
    explore_arms = int(x / 6)
    count += explore_arms
    a = candidates.CourseID.to_list()
    ex = list(np.random.choice(a, explore_arms, replace = False))
    temp = pd.DataFrame()
    for i in ex:
        b = candidates.loc[candidates['CourseID'] == i]
        df = pd.DataFrame({
            'CourseID': b.CourseID.values,
            'Rank': b.Rank.values
        })
        temp = pd.concat([temp, df])
    chosen_arms = pd.concat([chosen_arms, temp])
    candidates = candidates[~candidates.CourseID.isin(chosen_arms.CourseID)] # filter candidates
    remaining = x - count
    df_temp = pd.DataFrame({ 
        'CourseID':candidates.CourseID[: remaining],
        'Rank':candidates.Rank[: remaining]
    })
    chosen_arms = pd.concat([chosen_arms, df_temp])
    return chosen_arms

In [33]:
# input: test set, aggregated policy, x = number of recommendations to be obtained
def get_recommendations(test_df, corrected_policy, x):
    rec_df = pd.DataFrame()
    test_df_index = list(test_df.StudentID.unique())
    for i in test_df_index:
        student_data = test_df[test_df['StudentID'] == i].dropna(axis=0) 
        courses_taken = student_data.CourseID.to_list()
        candidates = corrected_policy[~corrected_policy.CourseID.isin(courses_taken)]
        recommendations = bandit(candidates, x)
        recommendations['StudentID'] = [i] * len(recommendations)
        rec_df = pd.concat([rec_df, recommendations])   
    return rec_df
        

In [98]:
pol_rec_binary =  get_recommendations(test_df_NaN_pol, corrected_policy, 10)
pol_rec_grades = get_recommendations(test_df_NaN_pol, corrected_policy_grades, 10)

In [99]:
pol_rec_binary_soft =  get_recommendations(test_df_NaN_pol, corrected_policy, 14)
pol_rec_grades_soft = get_recommendations(test_df_NaN_pol, corrected_policy_grades, 14)

In [81]:
pol_rec_binary.head(13)

,CourseID,Rank,StudentID
18,T207012,0.840278,266
21,T207004,0.823129,266
22,T206060,0.790541,266
23,T105001,0.784314,266
24,T207036,0.782609,266
26,T207034,0.751634,266
0,S102013,0.188811,266
0,T206057,0.246667,266
31,T207059,0.426471,266
32,12E051SCIENCES,0.362319,266


In [82]:
pol_rec_grades.head(11) # very slight differences

,CourseID,Rank,StudentID
17,T207012,4.157104,266
20,T207004,4.002674,266
22,T206060,3.945652,266
23,T105001,3.854037,266
24,T207036,3.849719,266
26,T207034,3.772606,266
0,T208000,0.870313,266
0,T206024,1.055901,266
31,T207059,2.125000,266
32,12E050SCIENCES,1.841837,266


In [41]:
# Save results, epsilon = 0.05, binary reward
pol_rec_binary.to_csv('C:/~your_path~/RecSys/obtained_data/eps05_binary_reward_recommendations_pol_bin.csv', index = False)
pol_rec_grades.to_csv('C:/~your_path~/RecSys/obtained_data/eps05_binary_reward_recommendations_pol_grades.csv', index = False)

In [38]:
complete_rec = get_recommendations(test_df, corrected_policy_complete, 10)

In [40]:
complete_rec.head(11)

,CourseID,Rank,StudentID
4,T207000,0.498813,313
5,T200003,0.496616,313
6,T100001,0.494841,313
7,T200000,0.493932,313
8,T207033,0.492225,313
0,7420AF,0.036364,313
9,T200002,0.490838,313
10,T207040,0.490597,313
11,T207041,0.489641,313
12,T100004,0.488000,313


## Testing diversity in Recommendations for StudentID 775036
### CF-based RecSys vs RL-based RecSys

In [42]:
cf_mean_rec = pd.read_csv('C:~your_path~/RecSys/obtained_data/rec_mean_grades.csv')
cf_maj_rec = pd.read_csv('C:~your_path~/RecSys/obtained_data/rec_majority.csv')

In [54]:
set(pol_rec_binary['StudentID'].unique()) & set(cf_maj_rec['StudentID'].unique())

{9, 18, 20, 45, 61, 92, 132, 189, 202, 220}

In [55]:
cf_mean = cf_mean_rec[cf_mean_rec.StudentID ==132].CourseID.values

In [56]:
cf_maj = cf_maj_rec[cf_maj_rec.StudentID == 132].CourseID.values

In [57]:
set(cf_mean) & set(cf_maj) # 6 courses in common

{'12E050SCIENCES',
 '12E051SCIENCES',
 'T207012',
 'T207034',
 'T207036',
 'T207059'}

In [58]:
rl_maj = pol_rec_binary[pol_rec_binary.StudentID == 132].CourseID.values

In [59]:
rl_maj

array(['T207012', 'T207004', 'T206060', 'T207036', 'T108000', 'T207059',
       'T207034', '12E051SCIENCES', '12E050SCIENCES', 'J2P201'],
      dtype=object)

In [60]:
rl_mean = pol_rec_grades[pol_rec_grades.StudentID == 132].CourseID.values

In [61]:
rl_mean

array(['T207012', 'T207004', 'T206060', 'T207036', 'T108000', 'T208014',
       'T207034', 'T207059', '12E050SCIENCES', '12E051SCIENCES'],
      dtype=object)

In [63]:
len(set(rl_mean) & set(rl_maj)) # 9 courses in common

9

In [64]:
len(set(rl_mean) & set(cf_mean)) # 6 courses in common

6

In [65]:
len(set(rl_maj) & set(cf_maj)) # 9 courses in common

9

In [66]:
comparison_mean = pd.DataFrame({
    'Bandit Rec Mean': rl_mean,
    'CF Rec Mean': cf_mean
})

comparison_maj = pd.DataFrame({
    'Bandit Rec Maj': rl_maj,
    'CF Rec Maj':cf_maj
})

In [68]:
comparison_maj # 9 courses in common

,Bandit Rec Maj,CF Rec Maj
0,T207012,T207012
1,T207004,T207004
2,T206060,T207036
3,T207036,T206060
4,T108000,T207034
5,T207059,T207059
6,T207034,12E051SCIENCES
7,12E051SCIENCES,12E050SCIENCES
8,12E050SCIENCES,J2D035
9,J2P201,J2P201


In [69]:
comparison_mean # 6 courses in common

,Bandit Rec Mean,CF Rec Mean
0,T207012,12E050SCIENCES
1,T207004,12E051SCIENCES
2,T206060,T208012
3,T207036,T208011
4,T108000,T208003
5,T208014,T208000
6,T207034,T207059
7,T207059,T207036
8,12E050SCIENCES,T207034
9,12E051SCIENCES,T207012


**$\rightarrow$ The recommendations obtained by ranking according explicit ratings (mean grades, or non-binary reward) display more diversity than ranking according to majority vote**

## Evaluation Binary Reward and Non-Binary Reward, Epsilon = 0.05 

In [70]:
# Evaluation in the same way as described in CF_Evaluation

def evaluate_rec(true_test_df, rec_df, entries_removed):
    evaluation_df = pd.DataFrame()
    IDs = rec_df['StudentID'].unique().tolist()
    for i in IDs:
        true = set(true_test_df[true_test_df['StudentID'] == i][-entries_removed:]['CourseID'].tolist())
        rec_temp = rec_df[rec_df['StudentID'] == i]
        rec = set(rec_temp['CourseID'].tolist())
        intersection = true & rec
        tp = len(intersection)
        fn = len(true) - len(intersection)
        fp = len(rec) - tp
        recall = tp / (tp + fn)
        precision = tp / (tp + fp)
        if (recall > 0) & (precision > 0): 
            f1 = (2*(precision * recall)) / (precision + recall)
        else: 
            f1 = 0
        evaluation_i = pd.DataFrame(columns = ['StudentID', 'Recall', 'Precision', 'F1'], index = [0])
        evaluation_i.iloc[0] = [i, recall, precision, f1]
        evaluation_df = pd.concat([evaluation_df, evaluation_i], axis = 0)
    evaluation_df = evaluation_df.set_index(['StudentID'])
    return evaluation_df

In [100]:
eval_test = evaluate_rec(test_df_pol, pol_rec_binary, 10)

In [101]:
eval_test_grades = evaluate_rec(test_df_pol, pol_rec_grades, 10)

In [102]:
eval_test_soft = evaluate_rec(test_df_pol, pol_rec_binary_soft, 10)
eval_test_grades_soft = evaluate_rec(test_df_pol, pol_rec_grades_soft, 10)

In [93]:
eval_test_complete = evaluate_rec(test_df, complete_rec, 10)

### Illustration of Results

### Hard Evaluation: 10 candidates for 10 recommendations

In [103]:

a = eval_test.mean() # Binary Political, eps = 0.05
b = eval_test_grades.mean() # Grades Political, eps = 0.05
c = eval_test_complete.mean() # Binary Both Programs 2, eps = 0.05


In [104]:
result_df_eps05 = pd.concat([a.rename('Binary Pol'), b.rename('Grades Pol'), c.rename('Binary Complete')], axis = 1)

In [105]:
result_df_eps05 = np.round(result_df_eps05 , 4)

In [106]:
result_df_eps05

,Binary Pol,Grades Pol,Binary Complete
Recall,0.4637,0.4719,0.0
Precision,0.4583,0.4667,0.0
F1,0.4609,0.4691,0.0


### Soft Evaluation: 14 candidates for 10 recommendations

In [107]:
a_soft = eval_test_soft.mean() # Binary Political Science, eps = 0.05
b_soft = eval_test_grades_soft.mean() # Grades Political Science, eps = 0.05
c_soft = eval_test_complete.mean() # Binary Both Programs 2, eps = 0.05

result_df_eps05_soft = pd.concat([a_soft.rename('Binary Pol'), b_soft.rename('Grades Pol'), c_soft.rename('Binary Complete')], axis = 1)
result_df_eps05_soft = np.round(result_df_eps05_soft , 4)
result_df_eps05_soft

,Binary Pol,Grades Pol,Binary Complete
Recall,0.5591,0.5659,0.0
Precision,0.3952,0.4000,0.0
F1,0.4630,0.4686,0.0


Independent on whether a soft (more candidates than recommendations) or a hard evaluation (candidates = recommendations) was conducted or whether the reward was binary or non-binary, the balanced mean between Recall and Precision remains at roughly 46 %. In contrast to the CF-based RecSys, a significant difference is apperent between using mean grades and majority vote, however, the F1-Measure remains at 46% for the majority vote approach (highest score)! Letting the bandit algorithm learn in a non-associative setting on both study programs at the same time, simply results in no useful recommendations. For having the scenario where the context is greater than 1, contextual bandits need to be deployed in order to obtain a valid recommendation policy.